In [0]:
# Retrieve the task value from the previous task (bronze), with a debugValue for local testing

bronze_output = dbutils.jobs.taskValues.get(taskKey="bronze", key="bronze_output", debugValue={"start_date": "", "bronze_adls": "", "silver_adls": ""})

silver_output = dbutils.jobs.taskValues.get(taskKey="silver", key="silver_output", debugValue={"start_date": "", "bronze_adls": "", "silver_adls": ""})


# Access individual variables
start_date = bronze_output.get("start_date", "'")
bronze_adls = bronze_output.get("bronze_adls", "")
silver_adls = bronze_output.get("silver_adls", "")
gold_adls = bronze_output.get("gold_adls", "")

#print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")
#print(f"Start Date: {start_date}, Gold ADLS: {gold_adls}")

print("bronze_output:", bronze_output)
print("silver_output:", silver_output)
#print("start_date:", start_date)
#print("gold_adls:", gold_adls)

bronze_output: {'start_date': '', 'bronze_adls': '', 'silver_adls': ''}
silver_output: {'start_date': '', 'bronze_adls': '', 'silver_adls': ''}


In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
#import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
df = spark.read.parquet(str(silver_output)).filter(col("time") > start_date)

In [0]:
df.show()  # Ensure df is loaded from a valid path or query, not from a dictionary.

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-7903648895083901>, line 1
----> 1 df.show()

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:1193, in DataFrame.show(self, n, truncate, vertical)
   1192 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1193     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:934, in DataFrame._show_string(self, n, truncate, vertical)
    917     except ValueError:
    918         raise PySparkTypeError(
    919             error_class="NOT_BOOL",
    920             message_parameters={
   (...)
    923             },
    924         )
    926 table, _ = DataFrame(
    927     plan.ShowString(
    928         child=self._plan,
    929         num_rows=n,
    930     

In [0]:
def get_country_code(lat, lon):

 try:
     coordinates = (float (lat), float(lon))
     result = rg.search(coordinates)[0].get('cc')
     print(f"Processed coordinates: {coordinates} -> {result}")
     return result
 
 except Exception as e:
     print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
     return None

In [0]:
get_country_code_udf = udf(get_country_code, StringType())

In [0]:
get_country_code_udf("48.8588443", "2.2943506")

Column<'get_country_code(48.8588443, 2.2943506)'>

In [0]:
df_with_location = \
                 df.\
                     withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [0]:
df_with_location_sig_class = \
                            df_with_location.\
                                        withColumn("sig_class", when(col("sig") < 100, "Low").\
                                                                when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                                                otherwise("High")
                                        )

In [0]:
df_with_location_sig_class.head()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8196687180840230>, line 1
----> 1 df_with_location_sig_class.head()

NameError: name 'df_with_location_sig_class' is not defined

In [0]:
# Final Gold output path (only CSV folder)
gold_output_path = f"{gold_adls}earthquake_events_gold_csv/"

# Write to CSV with header
df_with_location_sig_class.write \
    .mode("overwrite") \
    .option("header", "true") \
    .format("csv") \


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8196687180840233>, line 2
      1 # Final Gold output path (only CSV folder)
----> 2 gold_output_path = f"{gold_adls}earthquake_events_gold_csv/"
      4 # Write to CSV with header
      5 df_with_location_sig_class.write \
      6     .mode("overwrite") \
      7     .option("header", "true") \
      8     .format("csv") \
      9     .save(gold_output_path)

NameError: name 'gold_adls' is not defined